In [1]:
import copy
import glob
import pickle
import warnings
from datetime import datetime, timedelta
from itertools import product
import joblib

import cartopy
import cartopy.crs as ccrs
import cartopy.feature
import cartopy.feature as cfeature
import cartopy.feature as cf
import cartopy.io.shapereader as shpreader
import matplotlib as mpl
import matplotlib.path as mpath
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shapely.geometry as sgeom
import xarray as xr
from scipy import stats
from scipy.spatial.distance import cdist
from shapely import geometry
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

import keras
from keras.models import Sequential,Model
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, AveragePooling2D, Dropout
from keras.utils import to_categorical
from keras.layers import LeakyReLU, ReLU
from keras.callbacks import EarlyStopping

import visualkeras

import cluster_analysis, narm_analysis, som_analysis

2022-09-28 09:51:02.647639: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-28 09:51:02.827573: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /glade/u/apps/ch/opt/mpt_fmods/2.25/intel/19.1.1:/glade/u/apps/ch/opt/mpt/2.25/lib:/glade/u/apps/opt/intel/2020u1/compilers_and_libraries/linux/lib/intel64:/glade/u/apps/ch/os/usr/lib64:/glade/u/apps/ch/os/usr/lib:/glade/u/apps/ch/os/lib64:/glade/u/apps/ch/os/lib
2022-09-28 09:51:02.827600: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on yo

### Changes:

#### Batch size: more stable weights
#### Learning rate
#### Scaling: (-1,1) (0,1)
#### LeakyReLu vs. ReLu
#### alpha

# Functions

In [2]:
def get_cold_indx(ds, mo_init=9, mo_end=2):
    """
    Extract indices for cold season.
    Grabbing Sept thru February init, for Oct thru March predictions.
    """
    dt_array = pd.to_datetime(ds['date_range'])
    # return dt_array
    return xr.where((dt_array.month>=mo_init) | (dt_array.month<=mo_end), True, False)

In [11]:
def build_model(input_shape,activation_function, num_classes=4,learning_rate = 0.001):
    model = Sequential()
    size_kernel = 3
    model.add(Conv2D(64, kernel_size=(size_kernel, size_kernel),activation='linear',
                     input_shape=input_shape,padding='same'))
    model.add(activation_function())
    model.add(AveragePooling2D((size_kernel, size_kernel),padding='same'))
    model.add(Conv2D(128, (size_kernel, size_kernel), activation='linear',padding='same'))
    model.add(activation_function())
    model.add(AveragePooling2D(pool_size=(size_kernel, size_kernel),padding='same'))
    model.add(Conv2D(256, (size_kernel, size_kernel), activation='linear',padding='same'))
    model.add(activation_function())  
    model.add(AveragePooling2D(pool_size=(size_kernel, size_kernel),padding='same'))
    model.add(Flatten())
    model.add(Dense(256, activation='linear'))
    model.add(activation_function())                  
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(),\
                  metrics=["accuracy"])
    return model

In [12]:
def train_model(model, X_train, y_train,X_test, y_test,batch_size = 64, epochs = 50,path_save=None,verbose=0):
    callback = EarlyStopping(monitor='loss', patience=3)
    history = model.fit(X_train, y_train, batch_size=batch_size,\
        epochs=epochs,verbose=verbose,validation_data=(X_test, y_test), callbacks=[callback])
    if path_save:
        model.save(path_save)    
    return model

# 1. Train only for week 2

## 1.1 Load data

In [13]:
path_models_cnn = '/glade/work/jhayron/Weather_Regimes/models/CNN/'
path_files = '/glade/work/jhayron/Weather_Regimes/ERA5/'
anoms_week_1 = xr.open_dataset(f'{path_files}era5_z500_anoms_mean_week1.nc')

In [14]:
week = 2

# Load weather regimes time series
serie_wr_week = np.load(f'{path_files}Serie_WR_Week{week}.npy')

# Make Y categorical
serie_wr_categorical = to_categorical(serie_wr_week,num_classes=4)


## 1.2 Which scaling is better?

### 1.2.1 Min-Max

In [15]:
Min = anoms_week_1.anom.min(dim='time')
Max = anoms_week_1.anom.max(dim='time')

scaled_x = (anoms_week_1.anom) / (Max - Min)
#Reshape X
scaled_x = scaled_x.data.reshape(-1, anoms_week_1.anom.shape[1],anoms_week_1.anom.shape[2], 1)
X_train, X_test, y_train, y_test = train_test_split(scaled_x, serie_wr_categorical, test_size=0.2, random_state=42)

In [16]:
## Train and test model
input_shape = X_train.shape[1:]
model_temp = build_model(input_shape, ReLU, num_classes=4,learning_rate=0.01)
model_temp = train_model(model_temp, X_train, y_train,X_test, y_test,
    batch_size = 200, epochs = 50, path_save=f'{path_models_cnn}CNN_v0_week{week}_MinMaxScaling.h5',
    verbose=1)

Epoch 1/50
3/3 [==============================] - 7s 2s/step - loss: 1.3858 - accuracy: 0.2638 - val_loss: 1.3683 - val_accuracy: 0.3636
Epoch 2/50
3/3 [==============================] - 6s 2s/step - loss: 1.3618 - accuracy: 0.3509 - val_loss: 1.3251 - val_accuracy: 0.3545
Epoch 3/50
3/3 [==============================] - 6s 2s/step - loss: 1.3107 - accuracy: 0.3532 - val_loss: 1.3148 - val_accuracy: 0.3455
Epoch 4/50
3/3 [==============================] - 6s 2s/step - loss: 1.2797 - accuracy: 0.4060 - val_loss: 1.2472 - val_accuracy: 0.3909
Epoch 5/50
3/3 [==============================] - 6s 2s/step - loss: 1.2398 - accuracy: 0.4450 - val_loss: 1.2356 - val_accuracy: 0.3818
Epoch 6/50
3/3 [==============================] - 6s 2s/step - loss: 1.1922 - accuracy: 0.5161 - val_loss: 1.2382 - val_accuracy: 0.4182
Epoch 7/50
3/3 [==============================] - 6s 2s/step - loss: 1.1364 - accuracy: 0.4908 - val_loss: 1.2517 - val_accuracy: 0.4455
Epoch 8/50
3/3 [=========================

In [15]:
model_temp.evaluate(X_test,y_test)

4/4 [==============================] - 0s 45ms/step - loss: 5.3233 - accuracy: 0.4000


[5.323307991027832, 0.4000000059604645]

In [14]:
print(y_train)

[[0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 ...
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


In [ ]:

# Scale by standard deviation
standard_deviation = anoms_week_1.anom.std(dim='time')
scaled_x = anoms_week_1.anom / standard_deviation

In [ ]:
#Reshape X
scaled_x = scaled_x.data.reshape(-1, anoms_week_1.anom.shape[1],anoms_week_1.anom.shape[2], 1)

X_train, X_test, y_train, y_test = train_test_split(scaled_x, serie_wr_categorical, test_size=0.2, random_state=42)

# Create empty model
model = create_model()

# Train
callback = EarlyStopping(monitor='loss', patience=3)
batch_size = 64
epochs = 50
history = model.fit(X_train, y_train, batch_size=batch_size,\
    epochs=epochs,verbose=0,validation_data=(X_test, y_test), callbacks=[callback])
model.save(f'{path_models_cnn}CNN_v0_week{week}.h5')
joblib.dump(model,f'{path_models_cnn}CNN_v0_week{week}.history')

# Test
y_test_predicted = model.predict(X_test)
skill = 100 * len(np.where(np.argmax(y_test,axis=1) - np.argmax(y_test_predicted,axis=1) == 0)[0])/\
    len(np.argmax(y_test,axis=1))
skills.append(skill)
print(f'Skill week {week}: {np.round(skill,2)}%')
